# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [2]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean, pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head(5)

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
df_data = data.groupby(['CustomerID','ProductName'])['Quantity'].sum()

df_data.head()

CustomerID  ProductName                  
33          Apricots - Dried                 1
            Assorted Desserts                1
            Bandage - Flexible Neon          1
            Bar Mix - Pina Colada, 355 Ml    1
            Beans - Kidney, Canned           1
Name: Quantity, dtype: int64

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
table = pd.pivot_table(data, index=['ProductName'], 
               values=['Quantity'],
               columns=['CustomerID'],
               fill_value=0)
table.head()

Quantity                                \
CustomerID                           33    200   264   356   412   464     
ProductName                                                                
Anchovy Paste - 56 G Tube                0     0     0     0     0     0   
Appetizer - Mini Egg Roll, Shrimp        0     0     0     0     0     0   
Appetizer - Mushroom Tart                0     0     0     0     0     0   
Appetizer - Sausage Rolls                0     0     0     0     0     0   
Apricots - Dried                         1     0     0     0     1     0   

                                                           ...              \
CustomerID                        477   639   649   669    ... 97697 97753   
ProductName                                                ...               
Anchovy Paste - 56 G Tube             0     1     0     0  ...     0    25   
Appetizer - Mini Egg Roll, Shrimp     0     0     0     0  ...    25    25   
Appetizer - Mushroom Tart             0     1     0     0  ...    25     0   
Appetizer - Sausage Rolls             0     0     0     0  ...     0     0   
Apricots - Dried                      0     0     0     0  ...     0    25   

                                                                             \
CustomerID                        97769 97793 97900 97928 98069 98159 98185   
ProductName                                                                   
Anchovy Paste - 56 G Tube             0     0     0     0     0     0     0   
Appetizer - Mini Egg Roll, Shrimp     0     0     0     0     0     0     0   
Appetizer - Mushroom Tart             0     0     0     0     0     0    25   
Appetizer - Sausage Rolls             0     0    25    25    25     0    25   
Apricots - Dried                      0     0     0     0     0     0     0   

                                         
CustomerID                        98200  
ProductName                              
Anchovy Paste - 56 G Tube             0  
Appetizer - Mini Egg Roll, Shrimp     0  
Appetizer - Mushroom Tart             0  
Appetizer - Sausage Rolls             0  
Apricots - Dried                      0  

[5 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [7]:
sim_cust = squareform(pdist(table.T, 'euclidean'))
sim_cust

distances = pd.DataFrame(1/(1+squareform(pdist(table.T,'euclidean'))),
                         index=table.columns, columns=table.columns)
distances.head(5)

Quantity                                          \
CustomerID              33        200       264       356       412     
         CustomerID                                                     
Quantity 33          1.000000  0.085297  0.093953  0.091747  0.087410   
         200         0.085297  1.000000  0.085638  0.085297  0.080070   
         264         0.093953  0.085638  1.000000  0.088152  0.089301   
         356         0.091747  0.085297  0.088152  1.000000  0.085983   
         412         0.087410  0.080070  0.089301  0.085983  1.000000   

                                                                       ...  \
CustomerID              464       477       639       649       669    ...   
         CustomerID                                                    ...   
Quantity 33          0.089695  0.085297  0.088913  0.088152  0.089695  ...   
         200         0.083020  0.084959  0.083651  0.085638  0.087047  ...   
         264         0.087047  0.085638  0.086333  0.087047  0.087047  ...   
         356         0.086688  0.085983  0.091325  0.085983  0.087410  ...   
         412         0.085638  0.085638  0.089301  0.084959  0.087779  ...   

                                                                       \
CustomerID              97697     97753     97769     97793     97900   
         CustomerID                                                     
Quantity 33          0.004809  0.005108  0.004996  0.005421  0.004920   
         200         0.004825  0.005121  0.005014  0.005448  0.004925   
         264         0.004822  0.005115  0.004996  0.005441  0.004932   
         356         0.004814  0.005111  0.004999  0.005437  0.004920   
         412         0.004808  0.005131  0.004996  0.005441  0.004925   

                                                                       
CustomerID              97928     98069     98159     98185     98200  
         CustomerID                                                    
Quantity 33          0.005023  0.004880  0.005026  0.004549  0.004883  
         200         0.005032  0.004909  0.005042  0.004553  0.004879  
         264         0.005055  0.004894  0.005042  0.004566  0.004883  
         356         0.005036  0.004871  0.005042  0.004563  0.004886  
         412         0.005042  0.004876  0.005039  0.004568  0.004903  

[5 rows x 1000 columns]

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [12]:
customer_ID = '200'

customer = distances[customer_ID].sort_values(ascending=False)[1:6]


top5 = customer[:6].index.to_list()

# I don't know why but is giving my an KeyERROR that it shouldn't appear

KeyError: '200'

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [13]:
top5 = df_data[[3317,3305,3253,3472,2503]]
top5

CustomerID  ProductName                   
4261        Wine - Wyndham Estate Bin 777     2
            Soup Knorr Chili With Beans       2
3909        Scampi Tail                       1
4644        Fish - Scallops, Cold Smoked      2
2939        Bread - Italian Corn Meal Poly    1
Name: Quantity, dtype: int64

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [14]:
Aggregation = df_data.groupby(top5)['Quantity'].rank(ascending=False)

KeyError: 'Column not found: Quantity'

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.